In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)



In [1]:
!nvidia-smi

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [4]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)


In [6]:
%config Completer.use_jedi = False

In [5]:
model=keras.models.Sequential()

#32个卷积核*（3*3）个参数+32个偏置参数=320
model.add(keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=(28,28,1)))
#32个卷积核*（3*3）个参数*32个通道+32个偏置参数=9248
model.add(keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))

#64个卷积核*（3*3）个参数*32个通道+64个偏置参数=18496     
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
#64个卷积核*（3*3）个参数*64个通道+64个偏置参数=36928
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
          
#128个卷积核*（3*3）个参数*64个通道+128个偏置参数=73856 
model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
#128个卷积核*（3*3）个参数*128个通道+128个偏置参数=147584 
model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
          
model.add(keras.layers.Flatten())
#128*1152+128=147584  
model.add(keras.layers.Dense(128, activation='relu'))
#128*10+10=1290
model.add(keras.layers.Dense(10, activation="softmax"))
          
model.compile(loss="sparse_categorical_crossentropy",optimizer = "sgd",metrics = ["accuracy"])

In [6]:
model.summary()

In [18]:
!ls

In [ ]:
!rm -rf cnn-relu-callbacks

In [7]:
logdir = './cnn-relu-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"fashion_mnist_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,
                                    save_best_only = True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)


In [8]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

plot_learning_curves(history)

In [9]:
model.evaluate(x_test_scaled, y_test, verbose = 0)